In [ ]:
# Import necessary libraries
import cv2
import torch
from ultralytics import YOLO
import matplotlib.pyplot as plt
from collections import defaultdict

In [ ]:
# Load the model
model = YOLO("D:\Projects_Repo\Stealth_mode\model\\best.pt") 

<>:2: SyntaxWarning: invalid escape sequence '\P'
<>:2: SyntaxWarning: invalid escape sequence '\P'
C:\Users\hp\AppData\Local\Temp\ipykernel_9500\3357232907.py:2: SyntaxWarning: invalid escape sequence '\P'
  model = YOLO("D:\Projects_Repo\Stealth_mode\model\\best.pt")


In [ ]:
# Load the video
video_path = "D:\Projects_Repo\Stealth_mode\\video\\15sec_input_720p.mp4"
cap = cv2.VideoCapture(video_path)

<>:2: SyntaxWarning: invalid escape sequence '\P'
<>:2: SyntaxWarning: invalid escape sequence '\P'
C:\Users\hp\AppData\Local\Temp\ipykernel_9500\1191019509.py:2: SyntaxWarning: invalid escape sequence '\P'
  video_path = "D:\Projects_Repo\Stealth_mode\\video\\15sec_input_720p.mp4"
